In [13]:
import pandas as pd
# import dummy classifier
from sklearn.dummy import DummyClassifier
# import train_test_split
from sklearn.model_selection import train_test_split

In [14]:
data = pd.read_csv('train.csv')

In [15]:
data.head()


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,66728,21770,1,156,64.0,140,80,2,1,0,0,1,1
1,69098,21876,1,170,85.0,160,90,1,1,0,0,1,1
2,59185,23270,1,151,90.0,130,80,1,1,0,0,1,1
3,49288,19741,1,159,97.0,120,80,1,1,0,0,1,1
4,62481,18395,1,164,68.0,120,80,1,1,0,0,1,0


In [16]:
x_train, x_val, y_train, y_val = train_test_split(data.drop('cardio', axis=1), data['cardio'], test_size=0.2, random_state=0)

In [17]:
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(x_train, y_train)

dummy_clf.score(x_val, y_val)

0.4988392857142857

O KNN É um dos primeiros algoritmos aprendido no estudo de ML e é utilizado em problemas de classificação, ou sejam, tenta prever os resultados em uma saída discreta, em outras palavras, ele quer mapear variáveis ​​de entrada em categorias distintas.

o KNN tenta classificar cada amostra de um conjunto de dados avaliando sua distância em relação aos vizinhos mais próximos. Se os vizinhos mais próximos forem majoritariamente de uma classe, a amostra em questão será classificada nesta categoria.

In [21]:
#knn algorithm
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_clf.fit(x_train, y_train)

knn_clf.score(x_val, y_val)

0.6269642857142858

In [20]:
#import min max scaler (normalizar os dados), one hot encoder (transformar em binários os dados categóricos), and column transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# pode deixar dessa forma, se tiver mt ruim a gente muda
var_to_be_used = ['gluc', 'cholesterol', 'ap_hi', 'age', 'ap_lo', 'weight']
numerical_features = ['ap_hi', 'ap_lo', 'age', 'weight']
categorical_features = ['gluc', 'cholesterol']

# create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

x_train = preprocessor.fit_transform(x_train)
x_val = preprocessor.transform(x_val)

In [22]:
# import ransearchcv
from sklearn.model_selection import RandomizedSearchCV

# define the parameters
params = {
    'weights':['uniform', 'distance'],
    'n_neighbors': [3, 4, 5, 6, 7, 8],
    'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
}
# create the model
knn_clf = KNeighborsClassifier()

# create the random search cv
random_search = RandomizedSearchCV(knn_clf, param_distributions = params, n_iter = 5, scoring = 'roc_auc', n_jobs = -1, cv = 5, verbose = 3)

# fit the model
random_search.fit(x_train, y_train)

random_search.best_params_

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 2/5] END algorithm=kd_tree, n_neighbors=6, weights=uniform;, score=0.672 total time=   2.2s
[CV 2/5] END algorithm=auto, n_neighbors=4, weights=distance;, score=0.655 total time=   2.2s
[CV 1/5] END algorithm=auto, n_neighbors=4, weights=distance;, score=0.654 total time=   2.7s
[CV 1/5] END algorithm=kd_tree, n_neighbors=6, weights=uniform;, score=0.661 total time=   2.8s
[CV 4/5] END algorithm=auto, n_neighbors=4, weights=distance;, score=0.659 total time=   2.8s
[CV 3/5] END algorithm=auto, n_neighbors=4, weights=distance;, score=0.658 total time=   3.0s
[CV 3/5] END algorithm=kd_tree, n_neighbors=6, weights=uniform;, score=0.671 total time=   3.0s
[CV 5/5] END algorithm=auto, n_neighbors=4, weights=distance;, score=0.657 total time=   3.5s
[CV 4/5] END algorithm=kd_tree, n_neighbors=6, weights=uniform;, score=0.673 total time=   3.0s
[CV 2/5] END algorithm=ball_tree, n_neighbors=5, weights=uniform;, score=0.663 total t

{'weights': 'uniform', 'n_neighbors': 7, 'algorithm': 'auto'}

In [23]:
# train knn with the best params
knn_clf = KNeighborsClassifier(**random_search.best_params_)
knn_clf.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [24]:
knn_clf.score(x_val, y_val)

0.626875

In [25]:
data_test = pd.read_csv('test.csv')

In [34]:
data_test['cardio'].shape

(14000,)

In [46]:
x_test = preprocessor.transform(data_test)
y_test = knn_clf.predict(x_test)

In [47]:
from sklearn.metrics import accuracy_score
accuracy_score(data_test['cardio'], y_test)

0.6272142857142857